THƯ VIỆN

In [9]:
import os
import tensorflow.keras
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping

---

KHAI BÁO

In [3]:
dataset_dir = 'final_antispoofing'
train_dataset_dir = os.path.join(dataset_dir, 'train')
test_dataset_dir = os.path.join(dataset_dir, 'test')

In [4]:
need_dir = 'antispoofing_dataset'
train_dir = os.path.join(need_dir, 'train')
test_dir = os.path.join(need_dir, 'test')

In [5]:
catergories = ['real', 'spoof']

---

ĐỌC DATA

In [6]:
train_datagen = ImageDataGenerator(brightness_range=(0.8, 1.2), 
                                   rotation_range=30, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   fill_mode='nearest',
                                   shear_range=0.2,
                                   zoom_range=0.3,
                                   rescale=1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(160, 160), 
                                                    color_mode='rgb', 
                                                    class_mode='binary', 
                                                    batch_size=25, 
                                                    shuffle=True) 

valid_generator = train_datagen.flow_from_directory(test_dir, 
                                                    target_size=(160, 160), 
                                                    color_mode='rgb', 
                                                    class_mode='binary', 
                                                    batch_size=25, 
                                                    shuffle=True) 

Found 7481 images belonging to 2 classes.
Found 2737 images belonging to 2 classes.


---

TẠO MODEL

In [10]:
mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(160, 160, 3)))

In [11]:
mobilenet.trainable = False

In [12]:
output = Flatten()(mobilenet.output)
output = Dropout(0.3)(output)
output = Dense(units=8, activation='relu')(output)
prediction = Dense(1, activation='sigmoid')(output)

In [13]:
model = Model(inputs=mobilenet.input, outputs=prediction)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 160, 160, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 80, 80, 32)           864       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 80, 80, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 80, 80, 32)           0         ['bn_Conv1[0][0]']      

In [12]:
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(
        learning_rate=0.000001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07 
    ),
    metrics=['accuracy']
)

---

MODEL CHECKPOINT

In [15]:
model.load_weights('./model_weights_next/model_14-0.954128.h5')

In [16]:
model_checkpoint = ModelCheckpoint('./model_weights_next/model_last_{epoch:02d}-{val_accuracy:.6f}.h5', 
                                   monitor='val_loss', 
                                   mode='min',
                                   verboss=1,
                                   save_best_only=True,
                                   save_weights_only=True)

---

GET GO TRAINGING

In [17]:
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 25,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // 25,
    epochs=50,
    # callbacks=[model_checkpoint, early_stopping]
    callbacks=[model_checkpoint]
)

Epoch 1/50
299/299 [==============================] - 208s 668ms/step - loss: 0.1443 - accuracy: 0.9438 - val_loss: 0.1400 - val_accuracy: 0.9468
Epoch 2/50
299/299 [==============================] - 491s 2s/step - loss: 0.1288 - accuracy: 0.9509 - val_loss: 0.1382 - val_accuracy: 0.9472
Epoch 3/50
299/299 [==============================] - 87s 290ms/step - loss: 0.1292 - accuracy: 0.9512 - val_loss: 0.1212 - val_accuracy: 0.9549
Epoch 4/50
299/299 [==============================] - 85s 285ms/step - loss: 0.1353 - accuracy: 0.9502 - val_loss: 0.1442 - val_accuracy: 0.9472
Epoch 5/50
299/299 [==============================] - 397s 1s/step - loss: 0.1272 - accuracy: 0.9509 - val_loss: 0.1393 - val_accuracy: 0.9501
Epoch 6/50
299/299 [==============================] - 78s 262ms/step - loss: 0.1207 - accuracy: 0.9573 - val_loss: 0.1522 - val_accuracy: 0.9409
Epoch 7/50
299/299 [==============================] - 78s 262ms/step - loss: 0.1332 - accuracy: 0.9527 - val_loss: 0.1289 - val_accur

KeyboardInterrupt: 

---

LƯU CẤU TRÚC MODEL

In [15]:
load_model_to_json = '../model/ar.json'
model_json = model.to_json()
with open(load_model_to_json, 'w') as json_file:
    json_file.write(model_json)